In [10]:
# importar bibliotecas

import PyPDF2
import re
import sys
sys.path.insert(0, 'C:\\Users\\MatheusPereira\\OneDrive - Pontte\\Área de Trabalho\\automacaoRegistroCCI\\Scripts')
from lerContrato import lerContrato
import lerContrato as lc

In [11]:
contratoPath = r'C:\Users\MatheusPereira\OneDrive - Pontte\Área de Trabalho\automacaoRegistroCCI\Contratos\HE_Contrato_PatriciaMarcondes_Assinatura Digital_VFinal.pdf'

In [12]:
contrato = lerContrato(contratoPath)
quantidade = lc.dadosParticipantes(contratoPath, contrato)
contrato

{'valorTotal': '314917.04',
 'tabela': 'PRICE',
 'registro': '2766.67',
 'Taxa ao Mes': 0.1201,
 'valorLiquido': '290000.00',
 'prazoMes': '168',
 'valorPrimeiraParcelaComEncargos': '4038.78',
 'valorImóvel': '597000.00',
 'prazoContrato': '165',
 'ultimaParcela': '10/06/2036',
 'dataLiberação': '08/06/2022',
 'valorPrimeiraParcela': '3903.82',
 'primeiraParcela': '10/10/2022',
 'indice': 'IPCA',
 'Matrícula': '62.941',
 'Cartório': '02 de Registro Geral do 11º Oficial de Registro de Imóveis da Comarca de São Paulo/SP',
 'CCI': '0001031134/PCD',
 'Titular': 'PATRÍCIA CRUZ DE HAIDAR JORGE MARCONDES CABRAL ',
 'operação': 'PF',
 'Quantidade': 3,
 'Participante1': 'EDUARDO MARCONDES CABRAL',
 'Participação1': '65%',
 'Participante2': 'AMANDA CORAZZA MARCONDES CABRAL',
 'Participação2': '35%',
 'dataContrato': '06/06/2022'}

In [13]:
#Faz a leitura usando a biblioteca
pdf_file = open(contratoPath, 'rb')
read_pdf = PyPDF2.PdfFileReader(pdf_file)

# pega o numero de páginas
number_of_pages = read_pdf.getNumPages()

#Extriar Texto Página 1 a 5
text=''
for i in range(0,9):
    #Ler Página PDF
    pageObj = read_pdf.getPage(i)
    #Extrair Texto
    text=text+pageObj.extractText()

#Tratar Texto (Remover Quebra de Linhas e espaços duplos)
text = re.sub('\r', '', text) 
text = re.sub('\n', '', text)
text = re.sub(' {2,}', ' ', text).strip(' ')


In [5]:
listaKey = []
listaValues = []
if 'FI_' in contratoPath:
    # Campo participantes e agregados 
    campo3 = 'CAMPO 3 -'        # intervalo de extração 
    campo4 = 'CARACTERÍSTICAS DO FINANCIAMENTO'
    # Pegar posição das variáveis auxiliares no texto
    inicioTopico = text.find(campo3, 0)
    finalTopico = text.find(campo4, 0)

    # Criar Paragráfo Auxiliar (Somente com os sub itens do tópico 4. Despesas)
    campoPessoas= text[inicioTopico+len(campo3)+1:finalTopico-1]
    campoPessoas = re.sub('\s+',' ', campoPessoas)

    totalParticipantes = campoPessoas.count('NOME: ')
    totalParticipantes = totalParticipantes + campoPessoas.count('RAZÃO SOCIAL: ')
    listaKey.append('Quantidade')
    listaValues.append(totalParticipantes)
    print(campoPessoas)
    print(totalParticipantes)
if 'HE_' in contratoPath:
    # Campo participantes e agregados 
    campo2 = 'CAMPO 2 -'        # intervalo de extração 
    campo3 = 'CAMPO 3 -'
    # Pegar posição das variáveis auxiliares no texto
    inicioTopico = text.find(campo2, 0)
    finalTopico = text.find(campo3, 0)

    # Criar Paragráfo Auxiliar (Somente com os sub itens do tópico 4. Despesas)
    campoPessoas= text[inicioTopico+len(campo3)+1:finalTopico-1]
    campoPessoas = re.sub('\s+',' ', campoPessoas)

    totalParticipantes = campoPessoas.count('NOME: ')
    totalParticipantes = totalParticipantes + campoPessoas.count('RAZÃO SOCIAL: ')
    listaKey.append('Quantidade')
    listaValues.append(totalParticipantes)
    print(totalParticipantes)

3


In [6]:
if "HE_" in contratoPath:
    listaDePara = {'cpf':'CPF:','data de nascimento':'DATA DE NASCIMENTO:','endereço':'ENDEREÇO RESIDENCIAL E DOMICILIAR:','complemento':'COMPLEMENTO:','bairro':'BAIRRO:','cidade':'CIDADE:','uf':'UF:','cep':'CEP:',
                'telefone':'TELEFONE(S)','email':'EMAIL:'}

    #len(listaHE) # <--- Qtd de Itens na Lista
    listaKey = []
    listaValues = []

    # Remover cabeçalho da pg
    def removerText(valor):
        # Extraindo participantes da operação 
        começo = 'CONTRATO DE EMPRÉSTIMO'                        #inicio e fim da extração
        fim = 'Física Página'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = valor.find(começo, 0)
        finalTopico = valor.find(fim, 0)

        #Criar Paragráfo Auxiliar
        remover = valor[inicioTopico:finalTopico+len(fim)+8]
        remover = re.sub('\s+',' ', remover)
        return remover
        
    

    for qtdParticipantes in range(0, quantidade['Quantidade']):
        # Extraindo participantes da operação 
        começo = contrato[f'Participante{qtdParticipantes+1}']                        #inicio e fim da extração
        fim = 'CAMPO 3 -'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(começo, 0)
        finalTopico = text.find(fim, 0)

        #Criar Paragráfo Auxiliar
        paragrafoAux = text[inicioTopico+len(começo)+1:finalTopico-1]
        paragrafoAux = re.sub('\s+',' ', paragrafoAux)

        for key, value in listaDePara.items():
            if key == 'endereço':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'complemento':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("BAIRRO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'bairro':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("CIDADE:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'cidade':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("UF:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            elif key == 'telefone':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("EMAIL:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            else:
                paragrafoAux = paragrafoAux + ' '
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find(" ", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            listaKey.append(f'{key}{qtdParticipantes+1}')
            listaValues.append(valorExtraido.strip())
elif "FI_" in contratoPath:
    listaDePara = {'cpf':'CPF:','data de nascimento':'DATA DE NASCIMENTO:','endereço':'ENDEREÇO RESIDENCIAL E DOMICILIAR:','complemento':'COMPLEMENTO:','bairro':'BAIRRO:','cidade':'CIDADE:','uf':'UF:','cep':'CEP:',
                'telefone':'TELEFONE(S)','email':'EMAIL:'}

    #len(listaHE) # <--- Qtd de Itens na Lista
    listaKey = []
    listaValues = []
    for qtdParticipantes in range(0, quantidade['Quantidade']):
        # Extraindo participantes da operação 
        começo = contrato[f'Participante{qtdParticipantes+1}']                        #inicio e fim da extração
        fim = 'CARACTERÍSTICAS DO FINANCIAMENTO'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(começo, 0)
        finalTopico = text.find(fim, 0)

        #Criar Paragráfo Auxiliar
        paragrafoAux = text[inicioTopico+len(começo)+1:finalTopico-1]
        paragrafoAux = re.sub('\s+',' ', paragrafoAux)
        paragrafoAux
        for key, value in listaDePara.items():
            if key == 'endereço':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
            elif key == 'complemento':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("BAIRRO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
            elif key == 'bairro':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("CIDADE:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
            elif key == 'cidade':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("UF:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
            elif key == 'telefone':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("EMAIL:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco].strip()
            else:
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find(" ", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]

            listaKey.append(f'{key}{qtdParticipantes+1}')
            listaValues.append(valorExtraido.strip())


## Teste para extrair todos os participantes e agregados


In [22]:
if "HE_" in contratoPath:
    listaDePara = {'cpf':'CPF:','data de nascimento':'DATA DE NASCIMENTO:','endereço':'ENDEREÇO RESIDENCIAL E DOMICILIAR:','complemento':'COMPLEMENTO:','bairro':'BAIRRO:','cidade':'CIDADE:','uf':'UF:','cep':'CEP:',
                'telefone':'TELEFONE(S)','email':'EMAIL:'}

    #len(listaHE) # <--- Qtd de Itens na Lista
    listaKey = []
    listaValues = []
    partida = text.find('CAMPO 2 -', 0)
    for qtdParticipantes in range(0, contrato['Quantidade']):
        
        # Extraindo participantes da operação 
        começo = 'NOME:'                       #inicio e fim da extração
        fim = 'CAMPO 3 -'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(começo, partida)
        finalTopico = text.find(fim, 0)

        #Criar Paragráfo Auxiliar
        paragrafoAux = text[inicioTopico+len(começo)+1:finalTopico-1]
        paragrafoAux = re.sub('\s+',' ', paragrafoAux)
        for key, value in listaDePara.items():
            if key == 'endereço':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'complemento':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("BAIRRO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'bairro':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("CIDADE:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'cidade':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("UF:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            elif key == 'telefone':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("EMAIL:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            else:
                paragrafoAux = paragrafoAux + ' '
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find(" ", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            listaKey.append(f'{key}{qtdParticipantes+1}')
            listaValues.append(valorExtraido.strip())
        partida = (inicioTopico - partida) + partida + 5

#Criar Dicionario das duas Listas
dict_keyValue = dict(zip(listaKey,listaValues))
dict_keyValue


1456
PATRÍCIA CRUZ DE HAIDAR JORGE MARCONDES CABRAL CPF: 905.589.958-53 DATA DE NASCIMENTO: 10/02/1956 FILIAÇÃO: GASSEN DE HAIDAR JORGE e NATALINA CRUZ DE HAIDAR JORGE IDENTIDADE (tipo/número): RG: 7.940.084-X ÓRGÃO EMISSOR: SSP/SP NACIONALIDADE: BRASILEIRA PROFISSÃO: DO LAR ESTADO CIVIL: SEPARADA JUDICIALMENTE REGIME DE CASAMENTO (SE FOR CASADO): N/A ENDEREÇO RESIDENCIAL E DOMICILIAR: AVENIDA GIOVANNI GRONCHI, 4791 COMPLEMENTO: APTO 71, BL. A BAIRRO: VILA ANDRADE CONTRATO DE EMPRÉSTIMO (MÚTUO) E PACTO ADJETO DE ALIENAÇÃO FIDUCIÁRIA EM GARANTIA COM EMISSÃO DE CÉDULA DE CRÉDITO IMOBILIÁRIO - CCI CONTRATO Nº 0001031134/PCD REFIN – SFI RUBRICA DAS PARTES Contrato de Empréstimo (Mútuo) com Alienação Fiduciária – Pessoa Física Página 2 de 36 CIDADE: SÃO PAULO UF: SP CEP: 05724-003 TELEFONE(S): (11) 97373-4992 EMAIL: pc.hj@hotmail.com DEVEDORES SOLIDÁRIOS NOME: EDUARDO MARCONDES CABRAL CPF: 369.436.088-35 DATA DE NASCIMENTO: 04/08/1987 FILIAÇÃO: LECY MARCONDES CABRAL e PATRICIA CRUZ DE HAIDA

{'cpf1': '905.589.958-53',
 'data de nascimento1': '10/02/1956',
 'endereço1': 'AVENIDA GIOVANNI GRONCHI, 4791',
 'complemento1': 'APTO 71, BL. A',
 'bairro1': 'VILA ANDRADE',
 'cidade1': 'SÃO PAULO',
 'uf1': 'SP',
 'cep1': '05724-003',
 'telefone1': '(11) 97373-4992',
 'email1': 'pc.hj@hotmail.com',
 'cpf2': '369.436.088-35',
 'data de nascimento2': '04/08/1987',
 'endereço2': 'RUA VICENTE LEPORACE, 1331',
 'complemento2': 'APTO 102',
 'bairro2': 'CAMPO BELO',
 'cidade2': 'SÃO PAULO',
 'uf2': 'SP',
 'cep2': '04619-033',
 'telefone2': '(11) 97338-3626',
 'email2': 'eduardomcabral@gmail.com',
 'cpf3': '353.737.748-71',
 'data de nascimento3': '04/03/1986',
 'endereço3': 'RUA VICENTE LEPORACE, 1331',
 'complemento3': 'APTO 102',
 'bairro3': 'CAMPO BELO',
 'cidade3': 'SÃO PAULO',
 'uf3': 'SP',
 'cep3': '04619-033',
 'telefone3': '(11) 97468-9418',
 'email3': 'amanda.cmcabral@gmail.com'}

In [41]:
# Estrair dados do Titular

if contrato['operação'] == 'PJ':
    if "HE_" in contratoPath:
        listaDePara = {'endereçoTitular':'ENDEREÇO COMERCIAL:','complementoTitular':'COMPLEMENTO:','bairroTitular':'BAIRRO:','cidadeTitular':'CIDADE:','ufTitular':'UF:','cepTitular':'CEP:','cnpjTitular':'CNPJ:'}
        # Extraindo participantes da operação 
        começo = contrato['Titular'].strip()                    #inicio e fim da extração
        fim = 'CARACTERÍSTICAS DO FINANCIAMENTO'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(começo, 0)
        finalTopico = text.find(fim, 0)

        #Criar Paragráfo Auxiliar
        paragrafoAux = text[inicioTopico+len(começo)+1:finalTopico-1]
        paragrafoAux = re.sub('\s+',' ', paragrafoAux)
        
        for key, value in listaDePara.items():
            if key == 'endereçoTitular':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'complementoTitular':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("BAIRRO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'bairroTitular':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("CIDADE:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'cidadeTitular':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("UF:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            else:
                paragrafoAux = paragrafoAux + ' '
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find(" ", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
        
            listaKey.append(key)
            listaValues.append(valorExtraido.strip())


    elif 'FI_' in contratoPath:
        listaDePara = {'endereçoTitular':'ENDEREÇO COMERCIAL:','complementoTitular':'COMPLEMENTO:','bairroTitular':'BAIRRO:','cidadeTitular':'CIDADE:','ufTitular':'UF:','cepTitular':'CEP:','cnpjTitular':'CNPJ:'}
        # Extraindo participantes da operação 
        começo = contrato['Titular'].strip()                    #inicio e fim da extração
        fim = 'CARACTERÍSTICAS DO FINANCIAMENTO'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(começo, 0)
        finalTopico = text.find(fim, 0)

        #Criar Paragráfo Auxiliar
        paragrafoAux = text[inicioTopico+len(começo)+1:finalTopico-1]
        paragrafoAux = re.sub('\s+',' ', paragrafoAux)
        for key, value in listaDePara.items():
            if key == 'endereçoTitular':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'complementoTitular':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("BAIRRO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'bairroTitular':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("CIDADE:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'cidadeTitular':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("UF:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            else:
                paragrafoAux = paragrafoAux + ' '
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find(" ", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            
            listaKey.append(key)
            listaValues.append(valorExtraido.strip())   

elif contrato['operação'] == 'PF':
    if "HE_" in contratoPath:
        listaDePara = {'cpfTitular':'CPF:','data de nascimentoTitular':'DATA DE NASCIMENTO:','endereçoTitular':'ENDEREÇO RESIDENCIAL E DOMICILIAR:','complementoTitular':'COMPLEMENTO:','bairroTitular':'BAIRRO:','cidadeTitular':'CIDADE:','ufTitular':'UF:','cepTitular':'CEP:',
                'telefoneTitular':'TELEFONE(S)','emailTitular':'EMAIL:'}
        # Extraindo participantes da operação 
        começo = contrato['Titular'].strip()                    #inicio e fim da extração
        fim = 'CARACTERÍSTICAS DO FINANCIAMENTO'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(começo, 0)
        finalTopico = text.find(fim, 0)

        #Criar Paragráfo Auxiliar
        paragrafoAux = text[inicioTopico+len(começo)+1:finalTopico-1]
        paragrafoAux = re.sub('\s+',' ', paragrafoAux)

        for key, value in listaDePara.items():
            if key == 'endereço':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'complemento':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("BAIRRO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'bairro':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("CIDADE:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'cidade':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("UF:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            elif key == 'telefone':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("EMAIL:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            else:
                paragrafoAux = paragrafoAux + ' '
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find(" ", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            listaKey.append(key)
            listaValues.append(valorExtraido.strip())
    elif 'FI_' in contratoPath:
        listaDePara = {'cpfTitular':'CPF:','data de nascimentoTitular':'DATA DE NASCIMENTO:','endereçoTitular':'ENDEREÇO RESIDENCIAL E DOMICILIAR:','complementoTitular':'COMPLEMENTO:','bairroTitular':'BAIRRO:','cidadeTitular':'CIDADE:','ufTitular':'UF:','cepTitular':'CEP:',
                'telefoneTitular':'TELEFONE(S)','emailTitular':'EMAIL:'}
        # Extraindo participantes da operação 
        começo = contrato['Titular'].strip()                    #inicio e fim da extração
        fim = 'CARACTERÍSTICAS DO FINANCIAMENTO'
        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(começo, 0)
        finalTopico = text.find(fim, 0)

        #Criar Paragráfo Auxiliar
        paragrafoAux = text[inicioTopico+len(começo)+1:finalTopico-1]
        paragrafoAux = re.sub('\s+',' ', paragrafoAux)

        for key, value in listaDePara.items():
            if key == 'endereço':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'complemento':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("BAIRRO:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'bairro':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("CIDADE:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')

            elif key == 'cidade':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("UF:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            elif key == 'telefone':
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find("EMAIL:", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            else:
                paragrafoAux = paragrafoAux + ' '
                inicioFrase = paragrafoAux.find(value,0)
                finalFrase = inicioFrase + len(value) + 1
                proximoEspaco = paragrafoAux.find(" ", finalFrase)
                valorExtraido = paragrafoAux[finalFrase:proximoEspaco]
                if 'CONTRATO DE EMPRÉSTIMO' in valorExtraido:
                    apagar = removerText(valorExtraido)
                    valorExtraido = valorExtraido.replace(apagar,'')
            listaKey.append(key)
            listaValues.append(valorExtraido.strip())



#Criar Dicionario das duas Listas
dict_keyValue = dict(zip(listaKey,listaValues))
dict_keyValue


{'cpf1': '369.436.088-35',
 'data de nascimento1': '04/08/1987',
 'endereço1': 'RUA VICENTE LEPORACE, 1331',
 'complemento1': 'APTO 102',
 'bairro1': 'CAMPO BELO',
 'cidade1': 'SÃO PAULO',
 'uf1': 'SP',
 'cep1': '04619-033',
 'telefone1': '(11) 97338-3626',
 'email1': 'eduardomcabral@gmail.com',
 'cpf2': '353.737.748-71',
 'data de nascimento2': '04/03/1986',
 'endereço2': 'RUA VICENTE LEPORACE, 1331',
 'complemento2': 'APTO 102',
 'bairro2': 'CAMPO BELO',
 'cidade2': 'SÃO PAULO',
 'uf2': 'SP',
 'cep2': '04619-033',
 'telefone2': '(11) 97468-9418',
 'email2': 'amanda.cmcabral@gmail.com',
 'cpfTitular': '905.589.958-53',
 'data de nascimentoTitular': '10/02/1956',
 'endereçoTitular': 'AVENIDA',
 'complementoTitular': 'APTO',
 'bairroTitular': 'VILA',
 'cidadeTitular': 'SÃO',
 'ufTitular': 'SP',
 'cepTitular': '05724-003',
 'telefoneTitular': '',
 'emailTitular': 'pc.hj@hotmail.com'}

In [ ]:
for key, value in listaDePara.items():
    if key == 'endereçoTitular':
        inicioFrase = paragrafoAux.find(value,0)
        finalFrase = inicioFrase + len(value) + 1
        proximoEspaco = paragrafoAux.find("COMPLEMENTO:", finalFrase)
        valorExtraido = paragrafoAux[finalFrase:proximoEspaco]